# 課題4: 評判分析

本課題ではAmazonに投稿された映画のレビュー(英語)を分析し、レビューがPositiveかNegativeかの判別を行います。

今回の課題のディレクトリには `Training_data` と `Test_data` が入っています。さらにその中には `pos` と `neg` というディレクトリがあり、Positiveなデータが `pos` に、Negativeなデータが `neg` に入っています。

- Training_data (positive用)、文章数 : 700
- Training_data (negative用)、文章数 : 700
- Test_data (positive用)、文章数 : 3
- Test_data (negative用)、文章数 : 3

（ ※学習用データ：1400、　テスト用データ：6、　合計 : 1406 の文章です）

`Training_data`を用いて機械学習を行い、その結果を元に、6つのTest dataがPositiveかNegativeかを判別してください。

レッスン9で学んだ内容を踏まえ、各セルに'#コメント'の内容を実行するコードを記入してください。

わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なモジュールの読み込み

In [44]:
from pathlib import Path
import re

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

## 2. Dataの読み込み

`Training_data` からデータファイルを読み込みます。

In [45]:
# Training_data の読み込み
datapath = Path('Amazon_review/Training_data')
review_pattern = re.compile(r'cv\d+')  # ファイル名が "cv数字"　で始まるファイル名かを調べる正規表現
test_size = 0.25

data_orig  = dict(neg=[], pos=[])
data_train = dict(neg=[], pos=[])    # 学習データ
data_verify  = dict(neg=[], pos=[])    # 検証データ

# data_origへのデータの読み込み(training用ﾃﾞｰﾀ辞書化)
for cls, mails in data_orig.items():
    # data_orig内の各pos、negへのデータの読み込み
    for path in (datapath / cls).iterdir():
        if review_pattern.match(path.name):
            with open(path, 'r', encoding='latin') as src:
                mails.append(src.read())
    print(f"data_orig[{cls}]: loaded {len(mails)} e-mails.")
    
    #自分用メモ：train/verify内の各pos、negへのデータの分割と格納
    data_train[cls], data_verify[cls] = train_test_split(mails, test_size=test_size)
    print(f"data_train[{cls}]:{len(data_train[cls])}   data_verify[{cls}]:{len(data_verify[cls])}\n")


data_orig[neg]: loaded 700 e-mails.
data_train[neg]:525   data_verify[neg]:175

data_orig[pos]: loaded 700 e-mails.
data_train[pos]:525   data_verify[pos]:175



同じようにしてテスト用のファイルを読み込みます。`train_test_split` は不要です。

In [46]:
# Test_data の読み込み
datapath_test = Path('Amazon_review/Test_data')
review_test_pattern = re.compile(r'amazon_review_\d+')  # ファイル名が "amazon_review_数字"　で始まるファイル名かを調べる正規表現
data_test  = dict(neg=[], pos=[])    # テストデータ

# data_testへのデータの読み込み(test用ﾃﾞｰﾀ辞書化)
for cls, mails in data_test.items():
    # 自分用メモ：data_test内の各pos、negへのデータの読み込み
    for path in (datapath_test / cls).iterdir():
        if review_test_pattern.match(path.name):
            with open(path, 'r', encoding='latin') as src:
                mails.append(src.read())
    print(f"{cls:>8}: loaded {len(mails)} reviews.")

     neg: loaded 3 reviews.
     pos: loaded 3 reviews.


In [47]:
# データの整形用の関数　get_values_and_targets　を定義する
#自分用メモ：negとposの混合データvalues、そのbool解答targetの作成
def get_values_and_targets(data):
    values = data['neg'] + data['pos']
    target = [True]*len(data['neg']) + [False]*len(data['pos'])
    target = np.array(target)
    return values, target

# data_trainに対して get_values_and_targets を実行する
'''
自分用メモ：negとpos明示的に分かれた辞書data_trainから
neg、pos混合の一次配列values_train(前半neg,後半pos)とその並びの
一次配列bool型解答is_negative_train(前半True,後半False)を作成
'''
values_train, is_negative_train = get_values_and_targets(data_train)


Negativeなテストデータのうち、最初のデータを表示してみましょう。

In [48]:
# 読み込んだファイルの中身を表示（テストデータ, Negative, 1つ目）
print(data_test["neg"][0])

Iâve never been a Marvel Cinematic Universe fan; back in May 2012, the midnight premiere of The Avengers before I became a huge cinema buff, the only addition of the series I saw was Iron Man, which I felt no desire to watch again. I only bought a ticket to The Avengers because some friends invited me. Afterward, virtually the whole world went nuts and called it the greatest motion picture ever, I however just saw horrific, nauseous action void of artistic purpose. While softer attitudes towards Marvel eventually came my way, people praising ridiculous junk food over quality art still sickens me.

Even today, Marvel representing the early bane of my taste in movies still affects my outlook upon Avengers: Infinity War. Like Rocket Raccoonâs mockery upon anybody different, Marvelâs corporate heads blare their red logo in its abused emotional manipulation, whilst their followers turn a deep azure hue in moral discouragement.

This incorporation of many characters turns toilsome on a

## 3. データの前処理と特徴ベクトルの作成

次に `data_train` のデータを基に、特徴ベクトルを作成します。特徴ベクトルは `CountVectorizer` を使います。

「記号の削除」と「3文字未満の単語の削除」も指定してください。

In [49]:
# データの前処理を指定して、特徴量ベクトルを作成
vocab = CountVectorizer(token_pattern=r'[a-zA-Z]{3,}').fit([data_train['neg'][0]])
vocab.get_feature_names()

['actress',
 'admire',
 'adults',
 'after',
 'against',
 'also',
 'and',
 'audience',
 'awful',
 'baby',
 'bacharach',
 'backdrop',
 'bad',
 'bargains',
 'because',
 'become',
 'bell',
 'bette',
 'beyond',
 'biggest',
 'bit',
 'blame',
 'blow',
 'blows',
 'boomer',
 'boy',
 'burt',
 'but',
 'call',
 'can',
 'cancer',
 'character',
 'comedy',
 'comes',
 'credits',
 'did',
 'doesn',
 'dolls',
 'don',
 'drugs',
 'during',
 'dying',
 'earth',
 'even',
 'ever',
 'every',
 'everything',
 'exploring',
 'fail',
 'fame',
 'film',
 'find',
 'first',
 'for',
 'frankly',
 'from',
 'front',
 'generally',
 'gentile',
 'getting',
 'god',
 'gratingly',
 'greatness',
 'grunt',
 'hammy',
 'have',
 'hear',
 'heavy',
 'her',
 'heroes',
 'hollywood',
 'hope',
 'how',
 'impossible',
 'inspire',
 'instantly',
 'isn',
 'its',
 'jackie',
 'jacqueline',
 'jokes',
 'just',
 'kids',
 'late',
 'laughter',
 'lead',
 'life',
 'like',
 'little',
 'loud',
 'luck',
 'lurid',
 'maybe',
 'midler',
 'might',
 'model',
 'm

In [50]:
# print() で特徴量データを表示
print(vocab.transform([data_train["neg"][0]]))

  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	2
  (0, 4)	1
  (0, 5)	1
  (0, 6)	12
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	2
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	2
  (0, 21)	1
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  :	:
  (0, 150)	1
  (0, 151)	1
  (0, 152)	1
  (0, 153)	3
  (0, 154)	1
  (0, 155)	1
  (0, 156)	2
  (0, 157)	6
  (0, 158)	1
  (0, 159)	3
  (0, 160)	1
  (0, 161)	3
  (0, 162)	2
  (0, 163)	1
  (0, 164)	1
  (0, 165)	1
  (0, 166)	1
  (0, 167)	4
  (0, 168)	1
  (0, 169)	1
  (0, 170)	2
  (0, 171)	1
  (0, 172)	1
  (0, 173)	4
  (0, 174)	1


In [51]:
# 特徴量を配列形式で表示
vocab    = CountVectorizer(binary=True, token_pattern=r'[a-zA-Z]{3,}')
features = vocab.fit_transform(values_train)

In [52]:
# 特徴量のshapeを確認
print(features.shape)

(1050, 30927)


## 4. フィルタアルゴリズムの作成

ロジスティックモデルを使ってください。

In [53]:
# ロジスティックモデルを採用してフィッティングを行う
model = LogisticRegression(solver='saga', max_iter=1000, random_state=539167).fit(features, is_negative_train)

data_verify の特徴量ベクトルを作成し、モデルに分類させてみましょう。

In [54]:
# data_verifyに対して get_values_and_targets を実行する
'''
自分用メモ：テスト用辞書data_verify(negとpos明示的に分かれたもの)から
neg、pos混合の一次配列values_test(前半neg,後半pos)とその並びの
一次配列bool型解答is_negative_test(前半True,後半False)を作成
'''
values_test, is_negative_test = get_values_and_targets(data_verify)

# モデルに分類させる
'''
自分用メモ：Training_data内ﾄﾚｰﾆﾝｸﾞﾃﾞｰﾀ(7.5割)を元に作成したﾓﾃﾞﾙで
Training_data内検証ﾃﾞｰﾀ(2.5割)のvalues_test(前半neg,後半pos)を
transform(特徴量配列形式化)してﾛｼﾞｽﾃｨｯｸﾓﾃﾞﾙpredictにかける
'''
pred_test = model.predict(vocab.transform(values_test))

正答率を確認してください。

In [55]:
# 正答率を確認する
#自分用メモ：Training_data内検証ﾃﾞｰﾀ(2.5割)の正解率
validation = (pred_test == is_negative_test)
size = validation.size
correct = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

285/350 correct (81.429%)


## 5. テストデータで予測を実行する

検証データを使った検証の結果、比較的高い正答率が確認できたと思います。

最後に、data_test の **カテゴリごとに** 特徴量ベクトルを作成し、モデルに分類させてみましょう。

In [56]:
# data_testのカテゴリごとに正答率を確認する
for char in ("neg","pos"):
    '''
    自分用メモ：
    ・辞書ｷｰ毎の情報読み込み
    ・bool解答の配列作成（negはTrue、posはFalse 繰り返しの中で、全てTrueか全てFalseとなる）
    ・Training_data内ﾄﾚｰﾆﾝｸﾞﾃﾞｰﾀ(7.5割)で作成したﾓﾃﾞﾙでposのみかnegのみの予想作成
    ・予想と解答をつき合わせ、合っていれば(True=TrueかFalse=False)Trueとする正誤配列作成
    ・配列のサイズ＝ﾃｽﾄﾃﾞｰﾀ個数を取得
    ・正誤配列からnotFalse(True)のみを読み出し、正解数を算出
    '''
    _valuse = data_verify[char]
    _is_negative = [(char == "neg")]*len(_valuse)
    _pred = model.predict(vocab.transform(_valuse))
    _valid = (_pred == _is_negative)
    _size = _valid.size
    _correct = np.count_nonzero(_valid)
    print(f"{char:>8s}: {_correct:>3d}/{_size:>3d} correct ({_correct*100/_size:.3f}%)")

     neg: 145/175 correct (82.857%)
     pos: 140/175 correct (80.000%)
